# Set working dir

In [66]:
import os 
import warnings
import platform

warnings.filterwarnings("ignore")
if "macOS" in platform.platform():
    os.chdir("/Users/erjo3868/repos/hypedsearch/hypedsearch")
# Running on Fiji
else:
    os.chdir("/scratch/Shares/layer/hypedsearch/hypedsearch")

# SQL

In [4]:
import sqlite3

# Connect to a new SQLite3 database (it will create a new file)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create a table
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER
);
''')

# Insert some sample data
cursor.executemany('''
INSERT INTO users (name, age) VALUES (?, ?);
''', [('Alice', 30), ('Bob', 25), ('Charlie', 35)])

# Create an index on the 'age' column
cursor.execute('''CREATE INDEX age_index ON users (age);''')

# Try to create the same index again (with IF NOT EXISTS)
try:
    cursor.execute('''CREATE INDEX age_index ON users (age);''')
except sqlite3.OperationalError as err: 
    raise err

# Fetch the indices from the sqlite_master table to verify
cursor.execute("SELECT name FROM sqlite_master WHERE type='index';")
indices = cursor.fetchall()

# Commit changes and close the connection
conn.commit()
conn.close()


TypeError: 'OperationalError' object is not callable

## Creating table from dataclass

In [ ]:
from dataclasses import asdict, dataclass, fields
from typing import Any, List
from src.sql_database import get_create_table_query_for_dataclass, Sqlite3Database, get_sql_columns_from_dataclass

def add_data_classes_to_table(dataclasses: List[Any], table_name: str, db: Sqlite3Database):
    sql_colms = get_sql_columns_from_dataclass(obj=dataclasses[0])
    values = ""
    for colm in sql_colms:
        values += f":{colm.name}, "
    values = values[:-2] # remove traling ', '

    dataclasses_as_dicts = [asdict(x) for x in dataclasses]
    _=db.cursor.executemany(
        f"INSERT INTO {table_name} VALUES({values})", dataclasses_as_dicts
    )
    _=db.connection.commit()

@dataclass
class Product:
    id: int
    name: str
    price: float
    
table_name = "products"
db_path = ":memory:"
obj = Product

dataclasses = [
    Product(id=0, name="1", price=2),
    Product(id=1, name="2", price=3)
]

# Create table
db = Sqlite3Database(db_path=db_path)
query = get_create_table_query_for_dataclass(obj=dataclasses[0], table_name=table_name, primary_key_colm="name")
db.execute_query(query=query)
add_data_classes_to_table(dataclasses=dataclasses, table_name=table_name, db=db)

db.tables()
rows = db.all_table_rows(table_name=table_name)
# Recreate objects from rows
[Product(**row) for row in rows]


['products']

[Product(id=0, name='1', price=2.0), Product(id=1, name='2', price=3.0)]

## Template DB

In [ ]:
from src.sql_database import Sqlite3Database

# Load template
db = Sqlite3Database(db_path=":memory:")
db.cursor

# Create a table
db.cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER
    );
""")

# Insert some sample data
db.cursor.executemany('''
INSERT INTO users (name, age) VALUES (?, ?);
''', [('Alice', 30), ('Bob', 25), ('Charlie', 35)])

db.tables()
db.indices()

# Create index
db.cursor.execute("CREATE INDEX age_index ON users (age);")


db.tables()
db.indices()

table_name = "users"
db.all_table_rows(table_name=table_name)
db.table_info(table_name=table_name)
db.row_count(table_name=table_name)

['users']

[]

['users']

['age_index']

[{'id': 1, 'name': 'Alice', 'age': 30},
 {'id': 2, 'name': 'Bob', 'age': 25},
 {'id': 3, 'name': 'Charlie', 'age': 35}]

[{'cid': 0,
  'name': 'id',
  'type': 'INTEGER',
  'notnull': 0,
  'dflt_value': None,
  'pk': 1},
 {'cid': 1,
  'name': 'name',
  'type': 'TEXT',
  'notnull': 0,
  'dflt_value': None,
  'pk': 0},
 {'cid': 2,
  'name': 'age',
  'type': 'INTEGER',
  'notnull': 0,
  'dflt_value': None,
  'pk': 0}]

[{'COUNT(*)': 3}]

# Pydantic

In [7]:
from pathlib import Path
from typing import Literal, Union
from typing_extensions import Annotated
from pydantic import BaseModel, BeforeValidator, computed_field, field_validator


# Before validation
def to_path(path: Union[str, Path]):
    return Path(path)

class Model(BaseModel):
    path: Annotated[Path, BeforeValidator(to_path)]

    # @field_validator("path", mode='before')
    # @classmethod
    # def capitalize(cls, value: str) -> str:
    #     return Path(value)
    
str_path = "notebooks/spectra.ipynb"
path_path = Path(str_path)

x = Model(path=str_path)
x.path

x = Model(path=path_path)
x.path


# How pydantic treats Literal


class Model(BaseModel):
    ion_type: Literal["y"]

try:
    Model(ion_type="b")
except:
    print("Didn't like ion_type 'b'")

# Computed fields
class Model(BaseModel):
    val: float

    @computed_field
    @property
    def val_2(self) -> int:
        return self.val + 2
    
Model(val=2.5)


PosixPath('notebooks/spectra.ipynb')

PosixPath('notebooks/spectra.ipynb')

Didn't like ion_type 'b'


Model(val=2.5, val_2=4.5)

In [12]:
# Using constants in type annotations
from src.constants import B_ION_TYPE, Y_ION_TYPE


class ProductIon(BaseModel):
    charge: int
    type: Literal[B_ION_TYPE, Y_ION_TYPE]

ProductIon(charge=2, type="y")

ProductIon(charge=2, type='y')

In [21]:
# Updating a field after initialization
class A(BaseModel):
    a: str
    b: int

x = A(a="a", b=1)
x.a = "b"
x

# What happens when it's set to wrong type
x.a = 1
x


A(a='b', b=1)

A(a=1, b=1)

In [25]:
# Add new field
class A(BaseModel):
    a: str
    
    def new_field(self, val):
        # This fails!
        self.b = val

        # setattr(self, "b", val)

x = A(a="1")
x 
x.new_field(2)
x

A(a='1')

ValueError: "A" object has no field "b"

## Optional fields 

In [29]:
from typing import Optional


class Position(BaseModel):
    inclusive_start: Optional[int] = None
    exclusive_end: Optional[int] = None


class Kmer(BaseModel):
    seq: str
    position: Position


x = Kmer(seq="AB")
print(x.position.inclusive_start)

None


## Type annotations with enum

In [ ]:
from enum import Enum
from pydantic import BaseModel
class SqlColumnTypes(Enum):
    SQL_INT = "INTEGER"
    SQL_TEXT = "TEXT"
    SQL_REAL = "REAL"
    SQL_PRIMARY_KEY = "PRIMARY KEY"

class SqlColumn(BaseModel):
    name: str
    colm_type: SqlColumnTypes


SqlColumn(name="blah", colm_type="PRIMARY KEY")
# SqlColumnDef(name="blah", colm_type="NOT PRIMARY KEY")
SqlColumn(name="blah", colm_type=SqlColumnTypes)

SqlColumnDef(name='blah', colm_type=<SqlColumnTypes.SQL_PRIMARY_KEY: 'PRIMARY KEY'>)

ValidationError: 1 validation error for SqlColumnDef
colm_type
  Input should be 'INTEGER', 'TEXT', 'REAL' or 'PRIMARY KEY' [type=enum, input_value=<enum 'SqlColumnTypes'>, input_type=EnumType]
    For further information visit https://errors.pydantic.dev/2.10/v/enum

# Abstract class methods

In [17]:
from abc import ABC, abstractmethod

class BaseClass(ABC):
    @staticmethod
    @abstractmethod
    def fcn_1_var(seq: str) -> str:
        pass

    @staticmethod
    @abstractmethod
    def fcn_2_var(
        seq: str,
        charge: int,
    ):
        pass

class MissingFcn(BaseClass):
    @staticmethod
    def fcn_1_var(seq):
        return seq + "ABC"

    @staticmethod
    def fcn_2_var(seq):
        return seq + "ABC"
    
MissingFcn()

In [ ]:
from abc import ABC, abstractmethod

class BaseClass(ABC):
    @abstractmethod
    def my_method(self, arg1: int, arg2: str) -> bool:
        pass

# Regex

In [36]:
import re

a = "A"
b = "CABDDDDABDDDDAB"
# pattern = re.escape(a)  # Escape any special characters in `a` to treat it as a literal string
matches = re.finditer(a, b)

[(match.start(), match.end()) for match in matches]

[(1, 2), (7, 8), (13, 14)]

# Classes

## __repr__ vs __str__

In [18]:
class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author
    
    def __repr__(self):
        return f"Book(title={repr(self.title)}, author={repr(self.author)})"
    
    def __str__(self):
        return f"'{self.title}' by {self.author}"
    
book = Book("1984", "George Orwell")

print(repr(book))  # Calls __repr__
print(str(book))   # Calls __str__

print(f"this is the {book}")

Book(title='1984', author='George Orwell')
'1984' by George Orwell
this is the '1984' by George Orwell
